#Install yolov8

In [2]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()


import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

#Install Roboflow Supervision

In [ ]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.21.0


# Inference


In [3]:
import cv2
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO
from keras import models, layers, optimizers, callbacks
import threading
import queue
import time
import os
import csv
from datetime import datetime
import logging

# Load YOLOv8 model from Ultralytics hub
model = YOLO('/content/drive/MyDrive/Road_furniture_weights/content/runs/detect/train/weights/best.pt')  # Update with your model path

# Initialize video capture
video_path = '/content/drive/MyDrive/Copy of Start to End Hyderabad Part 3 720P.mp4'  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

# Open CSV file for writing
csv_file = '/content/drive/MyDrive/start-to-end-part3-info.csv'
csvfile = open(csv_file, 'w', newline='')  # Open the file object here
csv_writer = csv.DictWriter(csvfile, fieldnames=['object_id', 'timestamp', 'label', 'x', 'y', 'width', 'height'])
csv_writer.writeheader()

# Define object tracker class
class ObjectTracker:
    def __init__(self):
        self.objects = {}
        self.next_id = 0
        self.distance_threshold = 50  # Distance threshold for matching objects

    def update(self, box, label):
        center = (box[0], box[1])
        matched_id = None
        for obj_id, obj in self.objects.items():
            dist = np.linalg.norm(np.array(center) - np.array(obj['center']))
            if dist < self.distance_threshold:
                matched_id = obj_id
                break

        if matched_id is None:
            matched_id = self.next_id
            self.next_id += 1
            total_counts[label] += 1  # Only increment when a new object is found
            print(f"New object detected: {label} with ID {matched_id}")  # Debug print statement

        self.objects[matched_id] = {'center': center, 'label': label}
        return matched_id  # Return matched_id for object logging

# Initialize tracking history
track_history = defaultdict(lambda: deque(maxlen=30))
tracker = ObjectTracker()

total_counts = defaultdict(int)

# Object counts and frame tracking
object_counts = defaultdict(list)
frames = []
train_losses = []

# Data collection for training
X_train = []
y_train = []

# Define label mapping
label_mapping = {
    'cautionary': [1, 0, 0],
    'mandatory': [0, 1, 0],
    'informatory': [0, 0, 1]
}

# Define a neural network model for multi-class classification
nn_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(4,)),  # 4 features: x, y, width, height
    layers.Dense(32, activation='relu'),
    layers.Dense(3, activation='softmax')  # Three units for three classes
])
nn_model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Custom callback for real-time training loss visualization
class TrainingLossPlotter(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        train_losses.append(logs['loss'])

# Initialize the callback
loss_plotter = TrainingLossPlotter()

# Logging configuration
logging.basicConfig(filename='training.log', level=logging.INFO)

def train_model(X, y, queue):
    logging.info("Training started.")
    history = nn_model.fit(X, y, epochs=50, batch_size=32, callbacks=[loss_plotter], validation_split=0.2)
    queue.put(history)
    logging.info("Training completed.")

# Maintain a set of unique detected objects
unique_objects = set()

# Detection and tracking function
def detect_and_track(frame, frame_count, fps, csv_writer):
    global object_id_counter

    results = model(frame, conf=0.4, iou=0.4)  # Adjust confidence threshold as needed

    current_counts = defaultdict(int)
    boxes = results[0].boxes.xywh.cpu().numpy()  # Extract bounding boxes (x, y, width, height)
    class_ids = results[0].boxes.cls.cpu().numpy()  # Extract class labels

    frame_data = []
    frame_labels = []
    for box, class_id in zip(boxes, class_ids):
        label = model.names[int(class_id)]
        matched_id = tracker.update(box, label)

        # Calculate timestamp based on frame count and fps
        frame_time = frame_count / fps
        hours, remainder = divmod(frame_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        timestamp = f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

        # Create a unique identifier for the detected object
        unique_object_id = (matched_id, label)

        # Save to CSV for unique object
        if unique_object_id not in unique_objects:
            csv_writer.writerow({
                'object_id': matched_id,  # Use matched_id for object tracking
                'timestamp': timestamp,
                'label': label,
                'x': box[0],
                'y': box[1],
                'width': box[2],
                'height': box[3]
            })
            unique_objects.add(unique_object_id)  # Add to the set of unique objects
            print(f"Object logged to CSV: {label} with ID {matched_id} at timestamp {timestamp}")  # Debug print statement

        # Collect training data for all detections
        if label in label_mapping:
            frame_data.append([box[0], box[1], box[2], box[3]])
            frame_labels.append(label_mapping[label])

        # Update tracking history
        track = track_history[class_id]
        track.append((float(box[0]), float(box[1])))  # x, y center point

    frame_with_boxes = results[0].plot()  # Get the frame with bounding boxes

    return current_counts, frame_with_boxes, frame_data, frame_labels, results  # Return results here

# Read video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Explicitly set codec for compatibility
codec = cv2.VideoWriter_fourcc(*'mp4v')

# Output video writer
output_video_path = '/content/drive/MyDrive/start-end-part3-counts.mp4'  # Replace with your desired output path
out = cv2.VideoWriter(output_video_path, codec, fps, (frame_width, frame_height))

# Check if video writer is opened
if not out.isOpened():
    print("Error: Could not open output video file for writing.")
else:
    print("Output video file opened successfully.")

# Data collection and visualization loop
frame_count = 0
training_started = False
training_queue = queue.Queue()

# Resume from last processed frame
frame_count_file = '/content/drive/MyDrive/frame_count.txt'

if os.path.exists(frame_count_file):
    with open(frame_count_file, 'r') as f:
        frame_count = int(f.read().strip())
else:
    with open(frame_count_file, 'w') as f:
        f.write(str(frame_count))

cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection and tracking
    counts, frame_with_boxes, frame_data, frame_labels, results = detect_and_track(frame, frame_count, fps, csv_writer)  # Assign results here

    # Only add data if objects were detected
    if frame_data:
        X_train.extend(frame_data)
        y_train.extend(frame_labels)

    frames.append(frame_count)

    # Start training if we have enough data and training hasn't started yet
    if len(X_train) >= 100 and not training_started:
        X_train_np = np.array(X_train)
        y_train_np = np.array(y_train)
        training_thread = threading.Thread(target=train_model, args=(X_train_np, y_train_np, training_queue))
        training_thread.start()
        training_started = True

    # Plot the tracks
    for track_id, track in track_history.items():
        points = np.array(track).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(frame_with_boxes, [points], isClosed=False, color=(230, 230, 230), thickness=2)

    # Display total object counts on the frame
    y_pos = 50
    for label, count in total_counts.items():
        cv2.putText(frame_with_boxes, f'{label}: {count}', (10, y_pos), cv2.FONT_HERSHEY_SIMPLEX,  1, (0, 255,0 ), 3)
        y_pos += 30

    # Write the frame with boxes to the output video
    out.write(frame_with_boxes)
    print(f"Frame {frame_count} written to output video.")  # Debug print statement

    frame_count += 1

    # Save frame_count to file
    with open(frame_count_file, 'w') as f:
        f.write(str(frame_count))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Wait for training to complete if it has started
if training_started:
    training_thread.join()
    history = training_queue.get()
    print("Training completed.")

    # Save model weights
    nn_model.save_weights('model.weights.h5')

# Release video capture
cap.release()
out.release()
cv2.destroyAllWindows()

# Close CSV file
csvfile.close()

# Debug: Print final total counts
print("Final total counts:", dict(total_counts))


Output video file opened successfully.



0: 384x640 (no detections), 104.2ms
Speed: 16.6ms preprocess, 104.2ms inference, 117.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 3.1ms preprocess, 10.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 2.6ms preprocess, 10.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.0ms
Speed: 2.1ms preprocess, 11.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.8ms
Speed: 1.8ms preprocess, 10.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.2ms
Speed: 1.8ms preprocess, 11.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


Frame 0 written to output video.
Frame 1 written to output video.
Frame 2 written to output video.
Frame 3 written to output video.
Frame 4 written to output video.



0: 384x640 (no detections), 10.8ms
Speed: 2.3ms preprocess, 10.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 2.3ms preprocess, 10.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.5ms
Speed: 2.6ms preprocess, 11.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 2.2ms preprocess, 10.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.8ms
Speed: 2.5ms preprocess, 10.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)


Frame 5 written to output video.
Frame 6 written to output video.
Frame 7 written to output video.
Frame 8 written to output video.
Frame 9 written to output video.


...

Frame 9885 written to output video.
Frame 9886 written to output video.
Frame 9887 written to output video.


Speed: 2.0ms preprocess, 13.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 1.7ms preprocess, 10.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 1.7ms preprocess, 10.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 1.8ms preprocess, 10.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.6ms
Speed: 1.7ms preprocess, 11.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Frame 9888 written to output video.
Frame 9889 written to output video.
Frame 9890 written to output video.
Frame 9891 written to output video.
Frame 9892 written to output video.



0: 384x640 (no detections), 10.7ms
Speed: 1.8ms preprocess, 10.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.7ms
Speed: 1.8ms preprocess, 11.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.5ms
Speed: 1.8ms preprocess, 12.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 10.7ms
Speed: 1.7ms preprocess, 10.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.2ms
Speed: 1.8ms preprocess, 11.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Frame 9893 written to output video.
Frame 9894 written to output video.
Frame 9895 written to output video.
Frame 9896 written to output video.
Frame 9897 written to output video.
Final total counts: {'Cautionary': 26, 'Mandatory': 10, 'Informatory': 11}
